In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[9],7
commitnumber,1013f91,5014a19
conc,-,3.2e-07
dv,-,0.001
klin,-,2.22e-20
molecule,{'n2o': 3.2e-07},n2o
ng_adju,-,"[0, 0]"
ng_refs,-,"[2, 2]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.037020                    1 -0.012154
0.000750        2 -0.036781                    2 -0.012018
0.001052        3 -0.036237                    3 -0.011678
0.001476        4 -0.035653                    4 -0.011326
0.002070        5 -0.035061                    5 -0.010984
0.002904        6 -0.034479                    6 -0.010622
0.004074        7 -0.033927                    7 -0.010267
0.005714        8 -0.033382                    8 -0.009891
0.008015        9 -0.032878                    9 -0.009526
0.011243       10 -0.032419                   10 -0.009139
0.015771       11 -0.032029                   11 -0.008752
0.022122       12 -0.031741                   12 -0.008368
0.031031       13 -0.031600                   13 -0.007974
0.043528       14 -0.031622                   14 -0.007567
0.061057       15 -0.030354                   15 -0.006241
0.085645       16 -0.024805                   16 -0.002211
0.120136       17 -0.014869                   17  0.004680
0.168516       18 -0.002548                   18  0.013471
0.236378       19  0.012299                   19  0.024356
0.331549       20  0.030115                   20  0.037733
0.465100       21  0.051456                   21  0.054095
0.652400       22  0.077133                   22  0.073793
0.915100       23  0.100146                   23  0.091943
1.283650       24  0.097532                   24  0.093068
1.800600       25  0.072587                   25  0.077660
2.525700       26  0.048480                   26  0.059789
3.542800       27  0.030763                   27  0.043513
4.969550       28  0.019154                   28  0.029753
6.970850       29  0.012388                   29  0.018881
9.778100       30  0.008419                   30  0.010598
13.715850      31  0.005363                   31  0.004180
19.239350      32  0.002498                   32 -0.000683
26.987250      33  0.000262                   33 -0.004032
37.855300      34 -0.001606                   34 -0.006712
53.100050      35 -0.004150                   35 -0.010074
73.887500      36 -0.007532                   36 -0.014436
97.662500      37 -0.010719                   37 -0.018628
121.437500     38 -0.013565                   38 -0.022249
145.212500     39 -0.016960                   39 -0.025809
168.987500     40 -0.020367                   40 -0.028854
192.762500     41 -0.022017                   41 -0.030258
216.537500     42 -0.021678                   42 -0.029985
240.312500     43 -0.020868                   43 -0.029047
264.087500     44 -0.019921                   44 -0.027789
287.862500     45 -0.018807                   45 -0.026244
311.637500     46 -0.017505                   46 -0.024444
335.412500     47 -0.016012                   47 -0.022390
359.187500     48 -0.014336                   48 -0.020135
382.962500     49 -0.012488                   49 -0.017686
406.737500     50 -0.010490                   50 -0.015075
430.512500     51 -0.008386                   51 -0.012348
454.287500     52 -0.006222                   52 -0.009555
478.062500     53 -0.004026                   53 -0.006687
501.837500     54 -0.001794                   54 -0.003784
525.612500     55  0.000494                   55 -0.000830
549.387500     56  0.002861                   56  0.002190
573.162500     57  0.005321                   57  0.005285
596.937500     58  0.007884                   58  0.008457
620.712500     59  0.010556                   59  0.011705
644.487500     60  0.013340                   60  0.015024
668.262500     61  0.016241                   61  0.018409
692.037500     62  0.019265                   62  0.021847
715.812500     63  0.022418                   63  0.025378
739.587500     64  0.025703                   64  0.028964
763.362500     65  0.029111                   65  0.032581
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -20.873170  0.000000e+00 -20.873170 -20.697099   
0.000624    2     -20.873174  9.601783e-07 -20.873173 -20.697100   
0.000876    3     -20.873175  1.350754e-06 -20.873174 -20.697101   
0.001229    4     -20.873177  1.910207e-06 -20.873175 -20.697102   
0.001723    5     -20.873180  2.712853e-06 -20.873178 -20.697103   
0.002417    6     -20.873184  3.864185e-06 -20.873180 -20.697104   
0.003391    7     -20.873190  5.515217e-06 -20.873184 -20.697106   
0.004757    8     -20.873198  7.877792e-06 -20.873190 -20.697109   
0.006672    9     -20.873208  1.125737e-05 -20.873197 -20.697114   
0.009359    10    -20.873223  1.608668e-05 -20.873207 -20.697120   
0.013128    11    -20.873246  2.297849e-05 -20.873223 -20.697128   
0.018415    12    -20.873275  3.279705e-05 -20.873243 -20.697139   
0.025830    13    -20.873317  4.675715e-05 -20.873270 -20.697154   
0.036232    14    -20.873375  6.655325e-05 -20.873309 -20.697176   
0.050823    15    -20.873458  9.457458e-05 -20.873364 -20.697206   
0.071291    16    -20.873574  1.359947e-04 -20.873437 -20.697247   
0.100000    17    -20.873727  2.046116e-04 -20.873522 -20.697297   
0.140271    18    -20.873919  3.259716e-04 -20.873593 -20.697352   
0.196760    19    -20.874148  5.387045e-04 -20.873610 -20.697400   
0.275997    20    -20.874401  9.065724e-04 -20.873494 -20.697416   
0.387100    21    -20.874631  1.533633e-03 -20.873098 -20.697351   
0.543100    22    -20.874735  2.588060e-03 -20.872148 -20.697106   
0.761700    23    -20.874480  4.329751e-03 -20.870150 -20.696498   
1.068500    24    -20.873528  7.017835e-03 -20.866510 -20.695288   
1.498800    25    -20.871918  1.037914e-02 -20.861538 -20.693496   
2.102400    26    -20.870073  1.372435e-02 -20.856349 -20.691408   
2.949000    27    -20.868199  1.671186e-02 -20.851487 -20.689186   
4.136600    28    -20.866409  1.924931e-02 -20.847159 -20.686949   
5.802500    29    -20.864816  2.143689e-02 -20.843379 -20.684839   
8.139200    30    -20.863524  2.357361e-02 -20.839951 -20.683038   
11.417000   31    -20.862700  2.601865e-02 -20.836682 -20.681862   
16.014700   32    -20.862787  2.902630e-02 -20.833761 -20.682015   
22.464000   33    -20.864672  3.281952e-02 -20.831853 -20.684770   
31.510500   34    -20.869542  3.796979e-02 -20.831572 -20.692110   
44.200100   35    -20.879223  4.523774e-02 -20.833985 -20.707369   
62.000000   36    -20.897749  5.501245e-02 -20.842737 -20.736961   
85.775000   37    -20.931260  6.731146e-02 -20.863948 -20.789393   
109.550000  38    -20.973931  7.979556e-02 -20.894136 -20.854226   
133.325000  39    -21.025404  9.306562e-02 -20.932338 -20.929984   
157.100000  40    -21.086773  1.066705e-01 -20.980102 -21.016201   
180.875000  41    -21.158802  1.213392e-01 -21.037463 -21.111896   
204.650000  42    -21.239539  1.400705e-01 -21.099469 -21.214504   
228.425000  43    -21.325913  1.653939e-01 -21.160519 -21.321348   
252.200000  44    -21.416557  1.972681e-01 -21.219289 -21.431061   
275.975000  45    -21.510742  2.353481e-01 -21.275394 -21.542816   
299.750000  46    -21.607835  2.794760e-01 -21.328359 -21.655886   
323.525000  47    -21.707236  3.295788e-01 -21.377658 -21.769598   
347.300000  48    -21.808371  3.856168e-01 -21.422754 -21.883348   
371.075000  49    -21.910685  4.475568e-01 -21.463128 -21.996605   
394.850000  50    -22.013655  5.153548e-01 -21.498299 -22.108853   
418.625000  51    -22.116754  5.889110e-01 -21.527843 -22.219643   
442.400000  52    -22.219557  6.680961e-01 -21.551462 -22.328613   
466.175000  53    -22.321722  7.527370e-01 -21.568985 -22.435460   
489.950000  54    -22.422963  8.426408e-01 -21.580322 -22.539882   
513.725000  55    -22.523012  9.376361e-01 -21.585376 -22.641637   
537.500000  56    -22.621578  1.037594e+00 -21.583985 -22.740463   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')